<a href="https://colab.research.google.com/github/OwiyeD/Datawarehousing/blob/master/Water_Points_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraping water_points json dataset**



              ```
              {
              number_functional: …,
              number_water_points: {
              communityA: …,
              },
              community_ranking: …
              }

              ```

## Procedure

Summed all water_points with a status **"no"** in the column df['Water_functioning'] per the 65 community villages and divided the value with the total water_points per the respective community villages to find the percentage i.e. broken% = (df['Water_functioning']/water_points) * 100.
Then used the percentage brokage to rank the community where highest percentage brokage implied position 1 in the brokage rank while 0 percentage brokage implies position 65 since I used dense ranking but the max position











In [1]:
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize

with urllib.request.urlopen('https://raw.githubusercontent.com/onaio/ona-tech/master/data/water_points.json') as url:
    data = json.loads(url.read().decode())
    #I have to transform the json to dataframe since dataframe are optimized and have better analysis capabilities
    df = pd.DataFrame(json_normalize(data))
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,water_pay,respondent,research_asst_name,water_used_season,_bamboo_dataset_id,_deleted_at,water_point_condition,_xform_id_string,other_point_1km,_attachments,communities_villages,end,animal_number,water_point_id,start,water_connected,water_manager_name,_status,enum_id_1,water_lift_mechanism,districts_divisions,_uuid,grid,date,formhub/uuid,road_available,water_functioning,_submission_time,signal,water_source_type,_geolocation,water_point_image,water_point_geocode,deviceid,locations_wards,water_manager,water_developer,_id,animal_point,water_mechanism_plate,water_lift_mechanism_type,road_type,water_mechanism_plate_units,water_mechanism_plate_no,water_not_functioning,water_source_type_other,simserial,subscriberid
0,no,community,Haruna Mohammed,year_round,,None,functioning,_08_Water_points_CV,no,[north_ghana/attachments/1351696546452.jpg],Gumaryili,2012-11-12T11:46:32.454Z,more_500,xxx,2012-10-31T15:11:04.618Z,no,community members,submitted_via_web,5,no,northern,f8bcee72d7a0400fb99ae11bbf804010,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,no,yes,2012-11-13T07:13:57,low,dam_dugout,"[10.1892764, -0.66410362]",1351696546452.jpg,10.1892764 -0.66410362 155.10000610351563 5.0,355047040123780,west_mamprusi,community,community,381705,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no,community,Haruna Mohmmed,year_round,,None,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351701849971.jpg],Selinvoya,2012-11-12T11:49:36.619Z,50_to_500,xxx,2012-10-31T16:41:49.738Z,no,Amadu Salifu,submitted_via_web,5,yes,northern,c2f6b298955f47ab9f177bee1214141d,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13T07:14:04,high,unprotected_well,"[10.28173052, -0.56901122]",1351701849971.jpg,10.28173052 -0.56901122 201.89999389648438 5.0,355047040123780,west_mamprusi,individual,community,381706,yes,no,manual_power,gravel,NaN,NaN,NaN,NaN,NaN,NaN
2,no,community,Haruna Mohammed,year_round,,None,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351702462336.jpg],Selinvoya,2012-10-31T16:57:37.864Z,50_to_500,xxx,2012-10-31T16:52:02.601Z,no,Sulemana Abdulai,submitted_via_web,5,yes,northern,6bc6d188611d47f6a666cfd1eaa33998,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13T07:14:07,high,borehole,"[10.28169238, -0.56962993]",1351702462336.jpg,10.28169238 -0.56962993 202.60000610351563 5.0,355047040123780,west_mamprusi,community,individual,381707,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN
3,no,community,Haruna Mohammed,year_round,,None,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351702971561.jpg],Selinvoya,2012-10-31T17:06:55.047Z,50_to_500,xxx,2012-10-31T16:58:46.672Z,no,Haruna Mohammed,submitted_via_web,5,yes,northern,4b28ac4cbba744d79ba4257f772f94d6,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13T07:14:14,high,borehole,"[10.28115661, -0.56918339]",1351702971561.jpg,10.28115661 -0.56918339 199.6999969482422 5.0,355047040123780,west_mamprusi,individual,community,381708,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN
4,no,community,Haruna Mohammed,year_round,,None,functioning,_08_Water_points_CV,yes,[north_ghana/attachments/1351703622326.jpg],Selinvoya,2012-10-31T17:15:57.847Z,50_to_500,xxx,2012-10-31T17:08:27.160Z,no,Sulemana,submitted_via_web,5,yes,northern,7893ce5321804f229e533f36e90c9c6f,grid_further_500_m,2012-10-31,4d41d54d134c4bfa9078571addd819b9,yes,yes,2012-11-13T07:14:22,high,borehole,"[10.28044635, -0.56723556]",1351703622326.jpg,10.28044635 -0.56723556 208.6999969482422 5.0,355047040123780,west_mamprusi,community,community,381709,yes,no,manual_power,paved,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,no,community,Emmanuel y,year_round,,None,functioning,_08_Water_points_CV,no,[north_ghana/attachments/1358151218133.jpg],Suik,2013

In [2]:
#Let us list the columns
df.columns

Index(['water_pay', 'respondent', 'research_asst_name', 'water_used_season',
       '_bamboo_dataset_id', '_deleted_at', 'water_point_condition',
       '_xform_id_string', 'other_point_1km', '_attachments',
       'communities_villages', 'end', 'animal_number', 'water_point_id',
       'start', 'water_connected', 'water_manager_name', '_status',
       'enum_id_1', 'water_lift_mechanism', 'districts_divisions', '_uuid',
       'grid', 'date', 'formhub/uuid', 'road_available', 'water_functioning',
       '_submission_time', 'signal', 'water_source_type', '_geolocation',
       'water_point_image', 'water_point_geocode', 'deviceid',
       'locations_wards', 'water_manager', 'water_developer', '_id',
       'animal_point', 'water_mechanism_plate', 'water_lift_mechanism_type',
       'road_type', 'water_mechanism_plate_units', 'water_mechanism_plate_no',
       'water_not_functioning', 'water_source_type_other', 'simserial',
       'subscriberid'],
      dtype='object')

In [3]:
#How many unique communities do we have
communities = set(df.communities_villages)
#communities
print(len(communities))

65


In [4]:
#Sum the water points per community
water_points = df.groupby(['communities_villages'])['water_point_id'].size().to_frame(name='number_water_points').reset_index()
water_points

,communities_villages,number_water_points
0,Abanyeri,4
1,Akpari-yeri,3
2,Alavanyo,3
3,Arigu,12
4,Badomsa,27
...,...,...
60,Zogsa,6
61,Zua,28
62,Zuedema,18
63,Zukpeni,6


In [5]:
#Calculate how many water_points are functioning
functioning = df[df.water_functioning=='yes'].groupby(['communities_villages']).size().to_frame(name='number_functional').reset_index()
functioning

,communities_villages,number_functional
0,Abanyeri,4
1,Akpari-yeri,3
2,Alavanyo,2
3,Arigu,6
4,Badomsa,27
...,...,...
60,Zogsa,5
61,Zua,24
62,Zuedema,18
63,Zukpeni,2


In [6]:
#Broken water_points
broken = df[df.water_functioning=='no'].groupby(['communities_villages']).size().to_frame(name='broken').reset_index()
broken

,communities_villages,broken
0,Alavanyo,1
1,Arigu,6
2,Bandem,3
3,Banyangsa,3
4,Chanpolinsa,1
5,Dorinsa,1
6,Gbaarigu,3
7,Gbima,1
8,Guuta,2
9,Jagsa,6


In [ ]:
print(df['water_not_functioning'].value_counts())

kiosk_broken                              39
water_source_dry                          16
under_construction                        14
lift_broken                                7
missing_parts lift_broken kiosk_broken     4
improperly_constr                          2
lift_broken kiosk_broken                   1
kiosk_broken under_construction            1
water_source_dry under_construction        1
na_dn                                      1
poor_water other                           1
Name: water_not_functioning, dtype: int64


In [7]:
print(df['water_functioning'].value_counts())

yes      623
no        87
na_dn      2
Name: water_functioning, dtype: int64


In [10]:
from functools import reduce
import numpy as np

#List the sub-dataframes i.e. water_points, functioning and broken
dfs = [water_points, functioning, broken]
points_df = reduce(lambda left,right: pd.merge(left,right,how='left', on=['communities_villages']), dfs)
points_df.fillna(0, inplace=True)
points_df['brokage percent'] = [(x/y) *100 if x is not 0 else 0.0 for x,y in zip(points_df['broken'],points_df['number_water_points'])]
#points_df['broken%'] = points_df['broken%'].replace(0, 1)
points_df['rank'] = points_df['brokage percent'].rank(method='max',ascending=False)
points_df

,communities_villages,number_water_points,number_functional,broken,brokage percent,rank
0,Abanyeri,4,4,0.0,0.000000,65.0
1,Akpari-yeri,3,3,0.0,0.000000,65.0
2,Alavanyo,3,2,1.0,33.333333,9.0
3,Arigu,12,6,6.0,50.000000,4.0
4,Badomsa,27,27,0.0,0.000000,65.0
...,...,...,...,...,...,...
60,Zogsa,6,5,1.0,16.666667,18.0
61,Zua,28,24,4.0,14.285714,21.0
62,Zuedema,18,18,0.0,0.000000,65.0
63,Zukpeni,6,2,4.0,66.666667,1.0


In [11]:
commu = points_df[points_df['communities_villages'].isin([communities])]
commu


,communities_villages,number_water_points,number_functional,broken,brokage percent,rank


In [12]:
#Just confirm that broken water_points are actually 87
print(points_df['broken'].sum())

87.0


In [16]:
points_df.to_dict(orient='records')

[{'brokage percent': 0.0,
  'broken': 0.0,
  'communities_villages': 'Abanyeri',
  'number_functional': 4,
  'number_water_points': 4,
  'rank': 65.0},
 {'brokage percent': 0.0,
  'broken': 0.0,
  'communities_villages': 'Akpari-yeri',
  'number_functional': 3,
  'number_water_points': 3,
  'rank': 65.0},
 {'brokage percent': 33.33333333333333,
  'broken': 1.0,
  'communities_villages': 'Alavanyo',
  'number_functional': 2,
  'number_water_points': 3,
  'rank': 9.0},
 {'brokage percent': 50.0,
  'broken': 6.0,
  'communities_villages': 'Arigu',
  'number_functional': 6,
  'number_water_points': 12,
  'rank': 4.0},
 {'brokage percent': 0.0,
  'broken': 0.0,
  'communities_villages': 'Badomsa',
  'number_functional': 27,
  'number_water_points': 27,
  'rank': 65.0},
 {'brokage percent': 42.857142857142854,
  'broken': 3.0,
  'communities_villages': 'Bandem',
  'number_functional': 4,
  'number_water_points': 7,
  'rank': 6.0},
 {'brokage percent': 30.0,
  'broken': 3.0,
  'communities_vi